In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.image as mpimg
import os.path
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#creating a directory
image_dir = Path('../input/food41/images')

In [ ]:
class_names = sorted(os.listdir(image_dir))
n_classes = len(class_names)

In [ ]:
# Discover names of all 101 food classes in the dataset
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

## Creating File DataFrame

In [ ]:
#creating a Pandas dataframe
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)

category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
#checking how many photos are in each class - 100 as is expected
image_df['Label'].value_counts()

In [ ]:
# display the first 10 images and their labels
fig, axs = plt.subplots(2, 5, figsize=(10, 5))
axs = axs.flatten()

for i in range(10):
    img_name = image_df['Filepath'].iloc[i]
    img_label = image_df['Label'].iloc[i]
    
    img = cv2.imread(img_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    axs[i].imshow(img)
    axs[i].set_title(img_label)
    axs[i].axis('off')

plt.show()

## Train-Test Split

In [ ]:
#creating two image data generators - first one for train and validation datasets and the second one for test dataset
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
)

In [ ]:
# setup dataset

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
#defining a CNN using the MobileNetV2 architecture pre-trained on the ImageNet dataset
pretrained = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=[224,224,3], include_top=False, 
    weights='imagenet'
    )

pretrained.trainable = False

model = tf.keras.models.Sequential([
    pretrained,
    tf.keras.layers.GlobalAveragePooling2D(),
    layers.Dropout(0.25),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(n_classes, activation='softmax')
])

In [ ]:

#let's configure the learning process using Adam optimizer and
#look at the architecture of the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[ 'AUC']
)
print(model.summary())

In [ ]:
#training the model with a batch size of 64 and saving the training history for further analysis
EPOCHS = 50
BATCH_SIZE = 64
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.0001, patience=5, restore_best_weights = True)
history2 = model.fit(train_images,
                    steps_per_epoch=train_images.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=val_images,
                    validation_steps= val_images.samples // BATCH_SIZE // 2,
                    verbose=1,
                    callbacks=[early_stop],
                    shuffle=True)

In [ ]:
#assessing the model's performance on the test dataset
loss, auc = model.evaluate(test_images)

In [ ]:
#checking the available metrics
history_dict2 = history2.history

print(history_dict2.keys())

In [ ]:
def plot_train_instrumentation(epochs, data, train_param, val_param):
    
    plt.figure(figsize=(10,7))
    
    plt.plot(epochs, data[train_param], 'g', label=f'Training ({train_param})')
    plt.plot(epochs, data[val_param], 'red', label=f'Validation ({val_param})')
    
    plt.title("Training performance")
    plt.xlabel('Epochs')
    plt.ylabel(train_param)
    
    plt.legend()
    plt.show()

In [ ]:
#using the previously defined function to see the model's performance
#the loss functions don't converge in one point at the end 
#there is a need to apply data augmentation or tune hypermarameters, but so far this is the best version we've been able to get
epochs = range(1, len(history_dict2['auc'])+1)

plot_train_instrumentation(epochs, history_dict2, 'auc', 'val_auc')
plot_train_instrumentation(epochs, history_dict2, 'loss', 'val_loss')

In [ ]:
#let's generate a classification report for the predictions made by the trained model on the test dataset 
#to summarise the model's performance on each class
predictions = np.argmax(model.predict(test_images), axis=1)


report = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [ ]:

#let's look at the report
#the metrics vary between classes: the lowest F1 score is 0 for steak class and the highest is 0.97 for edamame class
print(report)